# Příklady použití

Zkusme mírně rozpracovat příklad s logováním: naše "decorator factory" teď bude záviset na globálne nastaveném `log_level`, podle kterého bude různě detailně zapisovat informace. Je to jen mírná modifikace předchozího příkladu.

In [ ]:
LOG_INFO    = 0
LOG_WARNING = 1
LOG_DEBUG   = 2

LOG_STR_LST = ["INFO", "WARNING", "DEBUG"]

log_level = LOG_DEBUG

def log(level = LOG_INFO):
    def dec(func):
        def wrapper(*args, **kwargs):
            if level <= log_level:
                print("{}: running function: {}".format(LOG_STR_LST[level], func.__name__))
                if log_level >= LOG_DEBUG:
                    print("\targs:", args)
                    print("\tkwargs:", kwargs)
            return func(*args, **kwargs)
        return wrapper
    return dec

@log(LOG_INFO)
def add(x, y):
    return x + y

@log(LOG_WARNING)
def do_warning_level_stuff():
    pass

@log(LOG_DEBUG)
def do_debug_level_stuff(**kwargs):
    pass

do_debug_level_stuff(neco = True)
add(1, 2)
do_warning_level_stuff()

## Měření času

Dekorátor je poměrně milý způsob, jak snadno k funkci přidat měření času. Použijme k tomu balík `time`, pomocí kterého si pouze zaznamenámě čas před a po spuštění měřené funkce.

In [ ]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        elapsed_time = end - start
        print(f"Elapsed time: {elapsed_time} seconds")
        return result
    return wrapper
    

## Fibonacciho čísla a memoizace

Naivní implementace výpočtu Fibonacciho čísel obvykle velmi rychle zaběhne do hluboké a široké rekurze, což je velmi pomalé. Demostrujme si to s použitím měřiče z předchozího příkladu.

In [ ]:
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
    
timed_fibonacci = timer(fibonacci)

timed_fibonacci(38)

Jednou možností, jak výpočet zrychlit, je přepsání pomocí obyčejného for cyklu s využitím takzvané memoizace - tedy ukládání výsledků předchozích běhů. K výpočtu n-tého Fibonacciho čísla potřebujeme vždy dvě předchozí. Abychom je nemuseli počítat pořád znovu, můžeme si je prostě uložit.

In [ ]:
@timer
def fibonacci_loop(n):
    a = 0
    b = 1
    for _ in range(1, n):
        a, b = b, a+b
    return b

fibonacci_loop(38)

V případě Fibonacciho čísel je to už takhle celkem jednoduché, ale můžeme zkusit napsat obecnější memoizaci pomocí dekorátoru. Zjednodušme si to předpokladem, že dekorovaná funkce bude přijímat jediný argument:

In [ ]:
def memoize(func):
    cache = {}
    def wrapper(n):
        if n in cache:
            return cache[n]
        result = func(n)
        cache[n] = result
        return result
    return wrapper


@memoize
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

timed_cached_fibonacci = timer(fibonacci)
timed_cached_fibonacci(38)